<a href="https://colab.research.google.com/github/pri-Mohanty/DeepDream_Mod/blob/main/DeepDream_Mod_%5E_%5E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Original DeepDream Algorithm**

In [ ]:
import tensorflow as tf
import numpy as np
import PIL.Image
import urllib.request
from moviepy.editor import ImageSequenceClip
import glob


# Load a pre-trained InceptionV3 model from TensorFlow Hub
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')
base_model.trainable = False

# Define a list of layers to target for feature visualization
layer_names = ['mixed3', 'mixed5', 'mixed7']

# Define the model that outputs the activation values for the target layers
dream_model = tf.keras.Model(inputs=base_model.input, outputs=[base_model.get_layer(name).output for name in layer_names])

# Define the target layer index for DeepDream
target_layer_index = layer_names.index('mixed5')

def calc_loss(img, model):
    # Calculate the activations of the target layers
    img_batch = tf.expand_dims(img, axis=0)
    layer_activations = model(img_batch)

    losses = []
    for act in layer_activations:
        # For each activation, maximize the mean of the activation values
        loss = tf.math.reduce_mean(act)
        losses.append(loss)

    return tf.reduce_sum(losses)

def calculate_cam(model, img, target_layer_index):
    """Calculates Class Activation Mapping (CAM)."""
    # Convert the image to a tensor if it is a numpy array
    if not tf.is_tensor(img):
        img = tf.convert_to_tensor(img, dtype=tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(img)
        layer_activations = model(img)
        target_activation = layer_activations[target_layer_index]
        loss = tf.reduce_mean(target_activation)
    grads = tape.gradient(loss, target_activation)
    weights = tf.reduce_mean(grads, axis=(0, 1, 2))  # Global average pooling
    cam = tf.reduce_sum(tf.multiply(weights, target_activation), axis=-1)
    cam = tf.nn.relu(cam)  # ReLU activation
    cam = cam / tf.reduce_max(cam)  # Normalize to [0, 1]
    return cam.numpy()

def visualize_cam(cam, img, alpha=0.5):
    """Overlays CAM onto the original image."""
    # Ensure cam is 2D
    if len(cam.shape) > 2:
        cam = cam.squeeze()
    # Resize CAM to match the original image dimensions
    cam = cv2.resize(cam, (img.shape[1], img.shape[0]))
    # Normalize CAM to [0, 255] and ensure values are within range
    cam = np.uint8(np.clip(cam * 255, 0, 255))
    # Convert CAM to heatmap
    heatmap = cv2.applyColorMap(cam, cv2.COLORMAP_JET)
    # Overlay heatmap on the original image
    heatmap = np.float32(heatmap) / 255
    cam_img = heatmap * alpha + img
    # Normalize the overlay image
    cam_img = cam_img / np.max(cam_img)
    return cam_img


def calculate_deepdream_score(model, img, target_layer_index):
    """Calculates DeepDream Score based on activation strength."""
    layer_activations = model(img)
    target_activation = layer_activations[target_layer_index]
    score = tf.reduce_mean(tf.abs(target_activation)).numpy()  # You can customize this calculation
    return score

def deepdream(img, model, steps=100, step_size=0.01):
    for step in range(steps):
        # Calculate the gradients of the image with respect to the loss
        with tf.GradientTape() as tape:
            tape.watch(img)
            loss = calc_loss(img, model)

        gradients = tape.gradient(loss, img)

        # Normalize the gradients
        gradients /= tf.math.reduce_std(gradients) + 1e-8

        # Update the image using gradient ascent
        img = img + gradients * step_size
        img = tf.clip_by_value(img, -1, 1)

        print(f"Step {step}: Gradients mean: {np.mean(gradients)}, Gradients max: {np.max(gradients)}")
        print(f"Step {step}: Loss: {loss}")

        # Capture CAM at intermediate steps
        if step % 2 == 0:
            # Convert img to NumPy array before visualizing
            img_np = img.numpy()
            cam = calculate_cam(model, np.expand_dims(img, axis=0), target_layer_index)
            cam_img = visualize_cam(cam, img_np.squeeze())
            cam_img = (cam_img * 255).astype(np.uint8)
            cam_img = Image.fromarray(cam_img)
            cam_img.save(f"cam_output_{step}.png")
            print(f"CAM visualization saved to cam_output_{step}.png")

    return img.numpy()

    return img

def run_deepdream(image_path, steps=100, step_size=0.01):
    # Load the input image
    img = tf.keras.preprocessing.image.load_img(image_path)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.applications.inception_v3.preprocess_input(img)

    # Convert to a tensor and run DeepDream
    img = tf.convert_to_tensor(img)
    img = deepdream(img, dream_model, steps, step_size)

    # Post-process the output image
    img = tf.keras.preprocessing.image.array_to_img(img)
    img.save('deepdream_output.jpg')

# Example usage
if __name__ == "__main__":
    image_url = "https://i0.wp.com/picjumbo.com/wp-content/uploads/man-looking-into-the-distance-on-top-of-the-mountain-free-photo.jpg?w=600&quality=80"
    urllib.request.urlretrieve(image_url, "input_image.jpg")
    run_deepdream("input_image.jpg", steps=50, step_size=0.01)
    image_files = sorted(glob.glob('cam_output_*.png'))
    clip = ImageSequenceClip(image_files, fps=15)
    clip.write_videofile("deepdream_cam_video.mp4")


# **DeepDream with Guided BackProp**

In [ ]:
!pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 31.8 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Fo

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import requests
import io
from google.colab import files
from moviepy.editor import ImageSequenceClip
import glob

# Load a pre-trained InceptionV3 model from TensorFlow Hub
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')
base_model.trainable = False

# Define a list of layers to target for feature visualization
layer_names = ['mixed3', 'mixed5', 'mixed7']

# Define the model that outputs the activation values for the target layers
dream_model = tf.keras.Model(inputs=base_model.input, outputs=[base_model.get_layer(name).output for name in layer_names])

# Define the target layer index for DeepDream
target_layer_index = layer_names.index('mixed7')

def guided_backprop(model, image, target_layer_index):
    with tf.GradientTape() as tape:
        tape.watch(image)
        layer_activations = model(image)
        target_activation = layer_activations[target_layer_index]
        loss = tf.math.reduce_mean(target_activation)

    grads = tape.gradient(loss, image)
    return grads


def deepdream(img, model, steps, step_size):
    img = tf.convert_to_tensor(img)
    for step in range(steps):
        grads = guided_backprop(model, img, target_layer_index)
        loss = tf.math.reduce_mean(model(img)[target_layer_index])
        grads = tf.clip_by_value(grads, -1, 1)  # Clip gradients to [-1, 1]
        img = tf.clip_by_value(img + grads * step_size, -1, 1)
        print(f"Step {step}: Gradients mean: {np.mean(grads)}, Gradients max: {np.max(grads)}")
        print(f"Step {step}: Loss: {loss}")

        # Capture CAM at intermediate steps
        if step % 2 == 0:
            # Convert img to NumPy array before visualizing
            img_np = img.numpy()
            cam = calculate_cam(model, img, target_layer_index)
            cam_img = visualize_cam(cam, img_np.squeeze())
            cam_img = (cam_img * 255).astype(np.uint8)
            cam_img = Image.fromarray(cam_img)
            cam_img.save(f"cam_output_{step}.png")
            print(f"CAM visualization saved to cam_output_{step}.png")

    return img.numpy()

def calculate_cam(model, img, target_layer_index):
    """Calculates Class Activation Mapping (CAM)."""
    # Convert the image to a tensor if it is a numpy array
    if not tf.is_tensor(img):
        img = tf.convert_to_tensor(img, dtype=tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(img)
        layer_activations = model(img)
        target_activation = layer_activations[target_layer_index]
        loss = tf.reduce_mean(target_activation)
    grads = tape.gradient(loss, target_activation)
    weights = tf.reduce_mean(grads, axis=(0, 1, 2))  # Global average pooling
    cam = tf.reduce_sum(tf.multiply(weights, target_activation), axis=-1)
    cam = tf.nn.relu(cam)  # ReLU activation
    cam = cam / tf.reduce_max(cam)  # Normalize to [0, 1]
    return cam.numpy()

def visualize_cam(cam, img, alpha=0.5):
    """Overlays CAM onto the original image."""
    # Ensure cam is 2D
    if len(cam.shape) > 2:
        cam = cam.squeeze()
    # Resize CAM to match the original image dimensions
    cam = cv2.resize(cam, (img.shape[1], img.shape[0]))
    # Normalize CAM to [0, 255] and ensure values are within range
    cam = np.uint8(np.clip(cam * 255, 0, 255))
    # Convert CAM to heatmap
    heatmap = cv2.applyColorMap(cam, cv2.COLORMAP_JET)
    # Overlay heatmap on the original image
    heatmap = np.float32(heatmap) / 255
    cam_img = heatmap * alpha + img
    # Normalize the overlay image
    cam_img = cam_img / np.max(cam_img)
    return cam_img

def calculate_sparsity(activations, threshold=0.1):
    """Calculates activation sparsity."""
    active_neurons = tf.cast(activations > threshold, tf.float32)
    sparsity = 1.0 - tf.reduce_mean(active_neurons).numpy()
    return sparsity

def calculate_deepdream_score(model, img, target_layer_index):
    """Calculates DeepDream Score based on activation strength."""
    layer_activations = model(img)
    target_activation = layer_activations[target_layer_index]
    score = tf.reduce_mean(tf.abs(target_activation)).numpy()  # You can customize this calculation
    return score


def download_and_preprocess_image(image_url):
    """Downloads an image from the URL and preprocesses it for DeepDream."""
    response = requests.get(image_url, stream=True)
    response.raise_for_status()
    # Decode the image data using Pillow
    img = Image.open(response.raw)
    img = img.convert('RGB')  # Ensure RGB format
    # Convert to numpy array and resize (optional)
    img = np.array(img)
    img = np.asarray(img, dtype='float32')
    # Manual Preprocessing (replace with your desired steps)
    img = img / 255.0  # Normalize pixel values to [0, 1]
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

def run_deepdream(image_url, steps=50, step_size=0.0001):
    img = download_and_preprocess_image(image_url)
    dream_img = deepdream(img, dream_model, steps=steps, step_size=step_size)
    dream_img = np.squeeze(dream_img, axis=0)  # Remove batch dimension
    dream_img = np.clip(dream_img, 0, 1)  # Clip values to [0, 1] range
    dream_img = (dream_img * 255).astype(np.uint8)  # Convert to uint8
    dream_img = Image.fromarray(dream_img)

    # Save the image to a file
    output_path = "deepdream_output.jpg"
    dream_img.save(output_path, format='JPEG')
    print(f"DeepDream image saved to {output_path}")

if __name__ == "__main__":
    image_url = "https://i0.wp.com/picjumbo.com/wp-content/uploads/man-looking-into-the-distance-on-top-of-the-mountain-free-photo.jpg?w=600&quality=80"
    run_deepdream(image_url, steps=4, step_size=0.0001)
    image_files = sorted(glob.glob('cam_output_*.png'))
    clip = ImageSequenceClip(image_files, fps=15)
    clip.write_videofile("deepdream_cam_video.mp4")


Step 0: Gradients mean: 4.780577267382569e-09, Gradients max: 0.001466822810471058
Step 0: Loss: 0.04755181819200516
CAM visualization saved to cam_output_0.png
Step 1: Gradients mean: 4.7738839548117085e-09, Gradients max: 0.0014655559789389372
Step 1: Loss: 0.04755200445652008
Step 2: Gradients mean: 4.7798875968396715e-09, Gradients max: 0.0014648305950686336
Step 2: Loss: 0.047552190721035004
CAM visualization saved to cam_output_2.png
Step 3: Gradients mean: 4.777386486409796e-09, Gradients max: 0.0014646705240011215
Step 3: Loss: 0.047552380710840225
DeepDream image saved to deepdream_output.jpg
Moviepy - Building video deepdream_cam_video.mp4.
Moviepy - Writing video deepdream_cam_video.mp4



Moviepy - Done !
Moviepy - video ready deepdream_cam_video.mp4


**Above code gave unremakable outputs.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **Adding total_variation_regularization**

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import requests
import urllib.request
import matplotlib.pyplot as plt
import cv2
from moviepy.editor import ImageSequenceClip
import glob

# Load a pre-trained InceptionV3 model from TensorFlow Hub
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')
base_model.trainable = False

# Define a list of layers to target for feature visualization
layer_names = ['mixed3', 'mixed5', 'mixed7']

# Define the model that outputs the activation values for the target layers
dream_model = tf.keras.Model(inputs=base_model.input, outputs=[base_model.get_layer(name).output for name in layer_names])

# Define the target layer index for DeepDream
target_layer_index = layer_names.index('mixed3')  # Change the target layer here if desired

# Define the regularization strength for total variation regularization
tv_reg_strength = 1e-6  # Adjust this value to control the smoothness

def guided_backprop(model, image, target_layer_index):
    with tf.GradientTape() as tape:
        tape.watch(image)
        layer_activations = model(image)
        target_activation = layer_activations[target_layer_index]
        loss = tf.math.reduce_mean(target_activation)

    grads = tape.gradient(loss, image)
    return grads

def deepdream(img, model, steps, step_size):
    img = tf.convert_to_tensor(img)
    for step in range(steps):
        grads = guided_backprop(model, img, target_layer_index)
        tv_loss = total_variation_regularization(img)
        loss = tf.math.reduce_mean(model(img)[target_layer_index]) + tv_reg_strength * tv_loss
        grads = tf.clip_by_value(grads, -1, 1)  # Clip gradients to [-1, 1]
        img = tf.clip_by_value(img + grads * step_size, -1, 1)
        print(f"Step {step}: Gradients mean: {np.mean(grads)}, Gradients max: {np.max(grads)}")
        print(f"Step {step}: Loss: {loss}")

        # Capture CAM at intermediate steps
        if step % 2 == 0:
            # Convert img to NumPy array before visualizing
            img_np = img.numpy()
            cam = calculate_cam(model, img, target_layer_index)
            cam_img = visualize_cam(cam, img_np.squeeze())
            cam_img = (cam_img * 255).astype(np.uint8)
            cam_img = Image.fromarray(cam_img)
            cam_img.save(f"cam_output_{step}.png")
            print(f"CAM visualization saved to cam_output_{step}.png")

    return img.numpy()


def total_variation_regularization(img):
    """Computes the total variation regularization loss for an image."""
    x_diff = img[:, :, :-1, :] - img[:, :, 1:, :]
    y_diff = img[:, :-1, :, :] - img[:, 1:, :, :]
    return tf.reduce_sum(tf.abs(x_diff)) + tf.reduce_sum(tf.abs(y_diff))

def download_and_preprocess_image(image_url):
    """Downloads an image from the URL and preprocesses it for DeepDream."""
    response = requests.get(image_url, stream=True)
    response.raise_for_status()
    # Decode the image data using Pillow
    img = Image.open(response.raw)
    img = img.convert('RGB')  # Ensure RGB format
    # Resize the image to the expected input size
    img = img.resize((299, 299), Image.ANTIALIAS)  # Resize to 299x299 pixels using ANTIALIAS resampling
    # Convert to numpy array
    img = np.array(img)
    img = np.asarray(img, dtype='float32')
    # Manual Preprocessing (replace with your desired steps)
    img = img / 255.0  # Normalize pixel values to [0, 1]
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    print(img.shape)
    return img

def calculate_cam(model, img, target_layer_index):
    """Calculates Class Activation Mapping (CAM)."""
    # Convert the image to a tensor if it is a numpy array
    if not tf.is_tensor(img):
        img = tf.convert_to_tensor(img, dtype=tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(img)
        layer_activations = model(img)
        target_activation = layer_activations[target_layer_index]
        loss = tf.reduce_mean(target_activation)
    grads = tape.gradient(loss, target_activation)
    weights = tf.reduce_mean(grads, axis=(0, 1, 2))  # Global average pooling
    cam = tf.reduce_sum(tf.multiply(weights, target_activation), axis=-1)
    cam = tf.nn.relu(cam)  # ReLU activation
    cam = cam / tf.reduce_max(cam)  # Normalize to [0, 1]
    return cam.numpy()

def visualize_cam(cam, img, alpha=0.5):
    """Overlays CAM onto the original image."""
    # Ensure cam is 2D
    if len(cam.shape) > 2:
        cam = cam.squeeze()
    # Resize CAM to match the original image dimensions
    cam = cv2.resize(cam, (img.shape[1], img.shape[0]))
    # Normalize CAM to [0, 255] and ensure values are within range
    cam = np.uint8(np.clip(cam * 255, 0, 255))
    # Convert CAM to heatmap
    heatmap = cv2.applyColorMap(cam, cv2.COLORMAP_JET)
    # Overlay heatmap on the original image
    heatmap = np.float32(heatmap) / 255
    cam_img = heatmap * alpha + img
    # Normalize the overlay image
    cam_img = cam_img / np.max(cam_img)
    return cam_img

def calculate_sparsity(activations, threshold=0.1):
    """Calculates activation sparsity."""
    active_neurons = tf.cast(activations > threshold, tf.float32)
    sparsity = 1.0 - tf.reduce_mean(active_neurons).numpy()
    return sparsity

def calculate_deepdream_score(model, img, target_layer_index):
    """Calculates DeepDream Score based on activation strength."""
    layer_activations = model(img)
    target_activation = layer_activations[target_layer_index]
    score = tf.reduce_mean(tf.abs(target_activation)).numpy()
    return score

def run_deepdream(image_url, steps, step_size):
    img = download_and_preprocess_image(image_url)
    print(f"Input image min: {np.min(img)}, Input image max: {np.max(img)}")
    dream_img = deepdream(img, dream_model, steps=steps, step_size=step_size)
    dream_img = np.squeeze(dream_img, axis=0)  # Remove batch dimension
    dream_img = np.clip(dream_img, 0, 1)  # Clip values to [0, 1] range
    dream_img = (dream_img * 255).astype(np.uint8)  # Convert to uint8
    dream_img = Image.fromarray(dream_img)
    cam = calculate_cam(dream_model, img, target_layer_index)
    deepdream_score = calculate_deepdream_score(dream_model, img, target_layer_index)
    print(f"DeepDream Score: {deepdream_score}")
    # Save the image to a file
    output_path = "deepdream_output.png"
    dream_img.save(output_path, format='PNG')
    print(f"DeepDream image saved to {output_path}")
    # Visualize and save the CAM
    cam_img = visualize_cam(cam, img.squeeze())  # Remove batch dimension from img
    cam_img = (cam_img * 255).astype(np.uint8)
    cam_img = Image.fromarray(cam_img)
    cam_img.save("final_cam_output.png")
    print("CAM visualization saved to final_cam_output.png")
    original_activations = dream_model(img)[target_layer_index]

    # Add back the batch dimension to dream_img for the model to work
    dream_img = tf.expand_dims(dream_img, axis=0)
    dream_activations = dream_model(tf.convert_to_tensor(dream_img))[target_layer_index]
    original_sparsity = calculate_sparsity(original_activations)
    dream_sparsity = calculate_sparsity(dream_activations)
    print(f"Original sparsity: {original_sparsity}, DeepDream sparsity: {dream_sparsity}")


if __name__ == "__main__":
  image_url = "https://i0.wp.com/picjumbo.com/wp-content/uploads/man-looking-into-the-distance-on-top-of-the-mountain-free-photo.jpg?w=600&quality=80"
  urllib.request.urlretrieve(image_url, "input_image.jpg")
  run_deepdream(image_url, steps=10, step_size=1.0)
  target_size = (299, 299)
  image_files = sorted(glob.glob('cam_output_*.png'))
  for image_file in image_files:
    img = Image.open(image_file)
    img = img.resize(target_size, Image.LANCZOS)
    img.save(image_file)
  clip = ImageSequenceClip(image_files, fps=15)
  clip.write_videofile("deepdream_cam_video.mp4")



  img = img.resize((299, 299), Image.ANTIALIAS)  # Resize to 299x299 pixels using ANTIALIAS resampling



(1, 299, 299, 3)
Input image min: 0.0, Input image max: 1.0
Step 0: Gradients mean: -6.127841345460183e-08, Gradients max: 0.0024878198746591806
Step 0: Loss: 0.4330541789531708
CAM visualization saved to cam_output_0.png
Step 1: Gradients mean: -6.302420985093704e-08, Gradients max: 0.0019795973785221577
Step 1: Loss: 0.4370095729827881
Step 2: Gradients mean: -6.564793864072271e-08, Gradients max: 0.0015474032843485475
Step 2: Loss: 0.4409131705760956
CAM visualization saved to cam_output_2.png
Step 3: Gradients mean: -6.687076137268377e-08, Gradients max: 0.0015458071138709784
Step 3: Loss: 0.44468024373054504
Step 4: Gradients mean: -6.843161770575534e-08, Gradients max: 0.0016221324913203716
Step 4: Loss: 0.448424369096756
CAM visualization saved to cam_output_4.png
Step 5: Gradients mean: -6.979712452448439e-08, Gradients max: 0.0015874376986175776
Step 5: Loss: 0.4521368443965912
Step 6: Gradients mean: -6.994440582275274e-08, Gradients max: 0.0016415618592873216
Step 6: Loss: 0

Moviepy - Done !
Moviepy - video ready deepdream_cam_video.mp4


# **Final Implementation**

In [ ]:
!pip install shap

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import requests
import urllib.request
import matplotlib.pyplot as plt
import cv2
from moviepy.editor import ImageSequenceClip
import glob
import shap
import requests
from io import BytesIO


# Load a pre-trained InceptionV3 model from TensorFlow Hub
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')
base_model.trainable = False

# Define a list of layers to target for feature visualization
layer_names = ['mixed3', 'mixed5', 'mixed7']

# Define the model that outputs the activation values for the target layers
dream_model = tf.keras.Model(inputs=base_model.input, outputs=[base_model.get_layer(name).output for name in layer_names])

# Define the target layer index for DeepDream
target_layer_index = layer_names.index('mixed3')  # Change the target layer here if desired

# Define the regularization strength for total variation regularization
tv_reg_strength = 1e-6  # Adjust this value to control the smoothness

def guided_backprop(model, image, target_layer_index):
    with tf.GradientTape() as tape:
        tape.watch(image)
        layer_activations = model(image)
        target_activation = layer_activations[target_layer_index]
        loss = tf.math.reduce_mean(target_activation)

    grads = tape.gradient(loss, image)
    return grads

def compute_feature_importance(model, image, target_layer_index):
    with tf.GradientTape() as tape:
        tape.watch(image)
        output = model(image)[target_layer_index]
        loss = tf.reduce_mean(output)
    gradients = tape.gradient(loss, image)
    feature_importance = tf.abs(gradients)
    return feature_importance

def compute_saliency_map(model, image, target_layer_index):
    with tf.GradientTape() as tape:
        tape.watch(image)
        layer_activations = model(image)
        target_activation = layer_activations[target_layer_index]
        loss = tf.reduce_mean(target_activation)
    grads = tape.gradient(loss, image)
    saliency_map = tf.reduce_max(tf.abs(grads), axis=-1, keepdims=True)
    saliency_map = tf.repeat(saliency_map, repeats=3, axis=-1)
    return saliency_map

def guided_grad_cam(model, image, target_layer_index):
    cam = calculate_cam(model, image, target_layer_index)
    guided_bp = guided_backprop(model, image, target_layer_index)

    # Resize CAM to match the input image size
    cam_resized = tf.image.resize(cam[..., tf.newaxis], (image.shape[1], image.shape[2]))

    # Expand dimensions to match guided backpropagation output
    cam_resized = tf.expand_dims(cam_resized, axis=0)
    cam_resized = tf.repeat(cam_resized, repeats=3, axis=-1)

    guided_grad_cam = cam_resized * guided_bp
    return guided_grad_cam

def adaptive_step_size(step, initial_step_size, decay_rate=0.9):
    return initial_step_size * (decay_rate ** step)

def deepdream(img, model, steps, initial_step_size, target_layer_index):
    img = tf.convert_to_tensor(img)
    for step in range(steps):
        with tf.GradientTape() as tape:
            tape.watch(img)
            loss = tf.reduce_mean(model(img)[target_layer_index])

        grads = tape.gradient(loss, img)
        grads = tf.math.l2_normalize(grads)

        feature_importance = compute_feature_importance(model, img, target_layer_index)
        feature_importance = tf.image.resize(feature_importance, (img.shape[1], img.shape[2]))
        feature_importance = tf.math.l2_normalize(feature_importance)

        saliency_map = compute_saliency_map(model, img, target_layer_index)
        saliency_map = tf.image.resize(saliency_map, (img.shape[1], img.shape[2]))
        saliency_map = tf.math.l2_normalize(saliency_map)

        ggcam = guided_grad_cam(model, img, target_layer_index)
        ggcam = tf.math.l2_normalize(ggcam)

        tv_loss = total_variation_regularization(img)

        # Ensure all guidance signals have the same shape as the input image
        combined_grads = grads + 0.2 * feature_importance + 0.2 * saliency_map + 0.2 * ggcam
        combined_grads = tf.clip_by_value(combined_grads, -1, 1)

        step_size = adaptive_step_size(step, initial_step_size)
        img = tf.clip_by_value(img + combined_grads * step_size, -1, 1)

        print(f"Step {step}: Loss: {loss}")

        # Capture CAM at intermediate steps
        if step % 2 == 0:
            img_np = img.numpy()
            cam = calculate_cam(model, img, target_layer_index)
            cam_img = visualize_cam(cam, img_np.squeeze())
            cam_img = (cam_img * 255).astype(np.uint8)
            cam_img = Image.fromarray(cam_img)
            cam_img.save(f"cam_output_{step}.png")
            print(f"CAM visualization saved to cam_output_{step}.png")

    return img.numpy()

def total_variation_regularization(img):
    """Computes the total variation regularization loss for an image."""
    x_diff = img[:, :, :-1, :] - img[:, :, 1:, :]
    y_diff = img[:, :-1, :, :] - img[:, 1:, :, :]
    return tf.reduce_sum(tf.abs(x_diff)) + tf.reduce_sum(tf.abs(y_diff))

def download_and_preprocess_image(image_url):
    """Downloads an image from the URL and preprocesses it for DeepDream."""
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(image_url, headers=headers)
    response.raise_for_status()
    img = Image.open(BytesIO(response.content))
    img = img.convert('RGB')
    img = img.resize((299, 299), Image.LANCZOS)
    img = np.array(img, dtype='float32') / 255.0
    img = np.expand_dims(img, axis=0)
    return img

def calculate_cam(model, img, target_layer_index):
    """Calculates Class Activation Mapping (CAM)."""
    if not tf.is_tensor(img):
        img = tf.convert_to_tensor(img, dtype=tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(img)
        layer_activations = model(img)
        target_activation = layer_activations[target_layer_index]
        loss = tf.reduce_mean(target_activation)
    grads = tape.gradient(loss, target_activation)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    cam = tf.reduce_sum(tf.multiply(pooled_grads, target_activation), axis=-1)
    cam = tf.nn.relu(cam)
    cam = cam / tf.reduce_max(cam)
    return cam[0].numpy()  # Return the first (and only) CAM as a numpy array

def visualize_cam(cam, img, alpha=0.5):
    """Overlays CAM onto the original image."""
    if tf.is_tensor(cam):
        cam = cam.numpy()
    if tf.is_tensor(img):
        img = img.numpy()
    if len(cam.shape) > 2:
        cam = np.squeeze(cam)
    # Ensure cam is 2D
    if len(cam.shape) != 2:
        raise ValueError(f"Expected CAM to be 2D, but got shape {cam.shape}")
    # Resize CAM to match the original image dimensions
    cam = cv2.resize(cam, (img.shape[1], img.shape[0]))
    # Normalize CAM to [0, 255] and ensure values are within range
    cam = np.uint8(np.clip(cam * 255, 0, 255))
    # Convert CAM to heatmap
    heatmap = cv2.applyColorMap(cam, cv2.COLORMAP_JET)
    # Overlay heatmap on the original image
    heatmap = np.float32(heatmap) / 255
    cam_img = heatmap * alpha + img
    # Normalize the overlay image
    cam_img = cam_img / np.max(cam_img)
    return cam_img


def calculate_sparsity(activations, threshold=0.1):
    """Calculates activation sparsity."""
    active_neurons = tf.cast(activations > threshold, tf.float32)
    sparsity = 1.0 - tf.reduce_mean(active_neurons).numpy()
    return sparsity

def calculate_diversity(activations):
    """Calculates activation diversity."""
    flat_activations = tf.reshape(activations, [activations.shape[0], -1])
    similarity = tf.matmul(flat_activations, flat_activations, transpose_b=True)
    diversity = 1 - tf.reduce_mean(similarity)
    return diversity.numpy()

def calculate_deepdream_score(model, img, target_layer_index):
    """Calculates DeepDream Score based on activation strength."""
    layer_activations = model(img)
    target_activation = layer_activations[target_layer_index]
    score = tf.reduce_mean(tf.abs(target_activation)).numpy()
    return score

def run_deepdream_multiscale(image_url, num_octaves=3, octave_scale=1.4, steps=10, initial_step_size=1.0):
    img = download_and_preprocess_image(image_url)
    base_shape = img.shape[1:3]
    for octave in range(num_octaves):
        new_shape = tuple(int(dim * (octave_scale ** octave)) for dim in base_shape)
        img = tf.image.resize(img, new_shape)
        img = deepdream(img, dream_model, steps=steps, initial_step_size=initial_step_size, target_layer_index=target_layer_index)

    dream_img = np.squeeze(img, axis=0)
    dream_img = np.clip(dream_img, 0, 1)
    dream_img = (dream_img * 255).astype(np.uint8)
    dream_img = Image.fromarray(dream_img)

    cam = calculate_cam(dream_model, img, target_layer_index)
    deepdream_score = calculate_deepdream_score(dream_model, img, target_layer_index)
    print(f"DeepDream Score: {deepdream_score}")

    output_path = "deepdream_output.png"
    dream_img.save(output_path, format='PNG')
    print(f"DeepDream image saved to {output_path}")

    cam_img = visualize_cam(cam, img.squeeze())
    cam_img = (cam_img * 255).astype(np.uint8)
    cam_img = Image.fromarray(cam_img)
    cam_img.save("final_cam_output.png")
    print("CAM visualization saved to final_cam_output.png")

    original_activations = dream_model(tf.convert_to_tensor(img))[target_layer_index]
    dream_activations = dream_model(tf.convert_to_tensor(img))[target_layer_index]

    original_sparsity = calculate_sparsity(original_activations)
    dream_sparsity = calculate_sparsity(dream_activations)
    print(f"Original sparsity: {original_sparsity}, DeepDream sparsity: {dream_sparsity}")

    original_diversity = calculate_diversity(original_activations)
    dream_diversity = calculate_diversity(dream_activations)
    print(f"Original diversity: {original_diversity}, DeepDream diversity: {dream_diversity}")

    return dream_img

def create_target_model(model, target_layer_index):
    return tf.keras.Model(inputs=model.inputs, outputs=model.outputs[target_layer_index])

target_model = create_target_model(dream_model, target_layer_index)

if __name__ == "__main__":
    image_url = "https://images.unsplash.com/photo-1494256997604-768d1f608cac?q=80&w=1229&auto=format&fit=crop&ixlib=rb-4.1.0&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D"
    run_deepdream_multiscale(image_url, num_octaves=3, octave_scale=1.4, steps=10, initial_step_size=1.0)

    target_size = (299, 299)
    image_files = sorted(glob.glob('cam_output_*.png'))
    for image_file in image_files:
        img = Image.open(image_file)
        img = img.resize(target_size, Image.LANCZOS)
        img.save(image_file)

    clip = ImageSequenceClip(image_files, fps=15)
    clip.write_videofile("deepdream_cam_video.mp4")

Step 0: Loss: 0.4329802989959717
CAM visualization saved to cam_output_0.png
Step 1: Loss: 0.4763002097606659
Step 2: Loss: 0.5150402784347534
CAM visualization saved to cam_output_2.png
Step 3: Loss: 0.5479326248168945
Step 4: Loss: 0.5754392147064209
CAM visualization saved to cam_output_4.png
Step 5: Loss: 0.5989159345626831
Step 6: Loss: 0.6188557744026184
CAM visualization saved to cam_output_6.png
Step 7: Loss: 0.6359995603561401
Step 8: Loss: 0.6510312557220459
CAM visualization saved to cam_output_8.png
Step 9: Loss: 0.6642836928367615
Step 0: Loss: 0.46253493428230286
CAM visualization saved to cam_output_0.png
Step 1: Loss: 0.5002923607826233
Step 2: Loss: 0.5295210480690002
CAM visualization saved to cam_output_2.png
Step 3: Loss: 0.5529636144638062
Step 4: Loss: 0.5724548697471619
CAM visualization saved to cam_output_4.png
Step 5: Loss: 0.5892603993415833
Step 6: Loss: 0.6037738919258118
CAM visualization saved to cam_output_6.png
Step 7: Loss: 0.6165904402732849
Step 8: L

Moviepy - Done !
Moviepy - video ready deepdream_cam_video.mp4
